## MSDI Project Notebook
Sample notebook for querying a MSDI feature service for Marine Protected Areas
### Todo:
* load into a Pandas dataframe for analysis
* 

In [7]:
from owslib.ogcapi.features import Features  #type: ignore
from datetime import datetime
from typing import List, TypedDict, Tuple, TypeAlias, Any, Dict, cast
from typing_extensions import Required

URI: TypeAlias = str

In [8]:
# Connect to the MPA feature service
wfs = Features('http://localhost/ogcfmsdi/')

# Set the /api root based on the OpenAPI specification
api = wfs.api()  # OpenAPI /api root

{'components': {'parameters': {'f': {'description': 'The optional f parameter indicates the output format which the server shall provide as part of the response document.  The default format is GeoJSON.', 'explode': False, 'in': 'query', 'name': 'f', 'required': False, 'schema': {'default': 'json', 'enum': ['json', 'html', 'jsonld'], 'type': 'string'}, 'style': 'form'}, 'lang': {'description': 'The optional lang parameter instructs the server return a response in a certain language, if supported.  If the language is not among the available values, the Accept-Language header language will be used if it is supported. If the header is missing, the default server language is used. Note that providers may only support a single language (or often no language at all), that can be different from the server language.  Language strings can be written in a complex (e.g. "fr-CA,fr;q=0.9,en-US;q=0.8,en;q=0.7"), simple (e.g. "de") or locale-like (e.g. "de-CH" or "fr_BE") fashion.', 'in': 'query', 'n

In [ ]:
from owslib.ogcapi.features import Features  #type: ignore
from datetime import datetime
from typing import List, TypedDict, Tuple, TypeAlias, Any, Dict, cast
from typing_extensions import Required, NotRequired
from shapely.geometry import shape  # type: ignore
from geojson.feature import Feature, FeatureCollection
from shapely.wkb import loads as wkbloads, dumps as wkbdumps  # type: ignore
from shapely.geometry.base import BaseGeometry  # type: ignore
import geojson #type: ignore

URI: TypeAlias = str

# Connect to the MPA feature service
wfs = Features('http://localhost/ogcfmsdi/')

# Set the /api root based on the OpenAPI specification
api = wfs.api()  # OpenAPI /api root
print(api)

In [ ]:

# determine the conformance of the service
conformance = wfs.conformance()
print("Conformance", conformance)




In [ ]:
# Fetch the set of collections available at the MSDI server.  The response format aligns with the Collections.yaml format.
# Each collection provides metadata information about the collection -  in this case, the feature collection of S-100 MPA features 
BBOX: TypeAlias = Tuple[float, float, float, float]

class ST_Extent(TypedDict, total = False):
    spatial: BBOX
    temporal: Tuple[ str, str]

class IETF_Link(TypedDict, total=False):
    "A link as per IETF RFC8288 represents a typed connection between two resources"
    link: str
    rel: str
    title: str
    href: str
    hreflang: str

class UOM(TypedDict, total=False):
    id: URI
    type: str
    code: str

class Parameter_(TypedDict, total=False):
    id: Required[str]
    type: str
    name: str
    encodingInfo: Dict[str, URI]
    nodata: bool | None
    uom: UOM
    _meta: Dict[str, Dict[str, str]]

class Collection_(TypedDict, total=False):
    id: Required[str]
    links: Required[List[IETF_Link]]
    title: str
    description: str
    keywords: List[str]
    attribution: str
    parameter_names: Dict[str, Dict[str, str]]  # ISSUE: The keyword is actually 'parameter-names' which breaks this class definition.
    extent: ST_Extent
    itemType: str
    crs: List[str]
    

class Collections_(TypedDict, total=False):
    collections: Required[Collection_]
    links: Required[List[IETF_Link]]
    timeStamp: datetime
    numberMatched: int
    numberReturned: int


In [ ]:
# Fetch the set of collection objects from the MPA features endpoint
collections: Tuple[Collection_, ...] = wfs.collections()
# print("Collections: \n\t", collections, "\n\n")
for k in collections:
        print(k)

for l in collections["collections"]:
    print("Collection:\n\t", l)
    
for l in collections["collections"]:
    print("\nCollection:\t", l.get('id'), "\n\tTitle:\t", l.get('title'), "\titemType:\t", l.get('itemType'), 
            "\n\tDescription:\t", l.get('description'), 
            "\n\tExtent:\t", l.get("extent"))


In [ ]:
OGC_POINT: TypeAlias = Tuple[float, float]

class OGC_Geometry(TypedDict, total=False ):
    type: Required[str]
    coordinates: Required[OGC_POINT | Tuple[OGC_POINT]]
    
class SF_Feature(TypedDict, total=False):
    type: Required[str]
    id: Any
    geometry: OGC_Geometry
    bbox: BBOX
    
class FixedDateRange(TypedDict, total=False):
    dateStart: str # datetime
    dateEnd: str # datetime

class FeatureName(TypedDict, total=False):
    name: str
    language: str


class MPA_Properties(TypedDict, total=False):
    fixedDateRange: FixedDateRange
    featureName: List[FeatureName]
    jurisdiction: str
    categoryOfMarineProtectedArea: str
    categoryOfRestrictedArea: str
    status: str

class MPA_Feature(Feature):
    def __init__(self, id=None, geometry=None, properties=None):
        super().__init__(id, geometry, properties)
        self._mpa_properties: MPA_Properties = properties
        
    @property
    def fixedDateRange(self)->FixedDateRange:
        try:
            return self._mpa_properties.get("fixedDateRange")
        except AttributeError as e:
            return FixedDateRange({"dateStart": None, "dateEnd": None})
    

# TODO: Implement a MPA_Feature JSONDecoder to deserialize the MPA into a proper feature class

class Items_(TypedDict, total=False):
    type: Required[str]
    features: Required[List[MPA_Feature]]
    timeStamp: datetime
    numberMatched: int
    numberReturned: int
    links: List[IETF_Link]
 

In [ ]:
# Request a view of all MPA features
# ISSUE: [MSDI] collection_items returns a Feature Collection albeit with foreign 
# The collection of MPA features is identified by the identifer 'S-100'
collectionID_ = 'S-122WDPA'
try:
    fc_: Items_ = wfs.collection_items(collection_id=collectionID_, limit=10)  
    for k in fc_:
        print(k)
        if k == 'type': print(fc_['type'])
    feature_count_: int = cast(int, fc_.get("numberReturned", 0))
    print('Collection:\t', collectionID_, 'Number of features:\t', feature_count_, '\n\n')
    # print('All features:\t', fc_)
    features = fc_.get("features")
    for cnt in range(feature_count_):
        f = features[cnt]
        # print(f)
        mpa: MPA_Feature = MPA_Feature(id=f.get("id"), geometry=f.get("geometry"), properties=f.get("properties"))

        print("Feature:\t", "ID:\t", mpa.id, "\n\t\tGeometry:\t", mpa.geometry, "\n\t\tProperties:\t", mpa.properties, "\n") 
except Exception as e:
    print(e)

